# Test Full Pipeline: End-to-End (Local)

This notebook runs the complete inspection pipeline end-to-end **locally**.

**Two pipeline modes** — toggle `USE_VISION_PIPELINE` in Cell 2:

| Mode | Extraction Method | GPU Required? | Speed |
|------|------------------|---------------|-------|
| **Vision** (NEW) | GPT-4o reads full drawing image | No (API only) | ~10-30s |
| **YOLO** (legacy) | YOLO detect → OCR → regex parse | Yes (~5.5 GB VRAM) | ~120-380s |

Both modes share the same downstream pipeline:
`normalize → validate → expand → match → score → assembly context → GPT-4o report`

**Requirements:**
- `.env` file with `OPENAI_API_KEY` (required for both modes)
- `.env` file with `HF_TOKEN` (required for YOLO mode only)
- Sample drawing image (PNG/JPG)
- Optional: SolidWorks JSON for comparison
- Optional: Assembly context databases

In [ ]:
# Cell 1: Setup — load .env, add project root to path
import sys
import os
from pathlib import Path

# Navigate to project root (two levels up from tests/notebooks/)
NOTEBOOK_DIR = Path(os.getcwd())
PROJECT_ROOT = NOTEBOOK_DIR
# Walk up until we find ai_inspector/
for _ in range(5):
    if (PROJECT_ROOT / 'ai_inspector').is_dir():
        break
    PROJECT_ROOT = PROJECT_ROOT.parent

# Add project root to Python path so ai_inspector is importable
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Load .env for HF_TOKEN and OPENAI_API_KEY
from dotenv import load_dotenv
load_dotenv(PROJECT_ROOT / '.env')

HF_TOKEN = os.getenv('HF_TOKEN')
if HF_TOKEN:
    print(f'HF_TOKEN loaded (length={len(HF_TOKEN)})')
else:
    print('WARNING: HF_TOKEN not found in .env — OCR model loading will fail.')

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if OPENAI_API_KEY:
    print(f'OPENAI_API_KEY loaded (length={len(OPENAI_API_KEY)})')
else:
    print('WARNING: OPENAI_API_KEY not found in .env — report generation will be skipped.')

import torch
print(f'Project root: {PROJECT_ROOT}')
print(f'Python: {sys.version}')
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

In [ ]:
# Cell 2: Configure paths — EDIT THESE for your test case
import os
from pathlib import Path

# ============================================================
# PIPELINE MODE — set True for GPT-4o vision, False for YOLO+OCR
# ============================================================
USE_VISION_PIPELINE = True

# ============================================================
# EDIT THESE PATHS to point to your test files
# ============================================================
SAMPLE_IMAGE = str(PROJECT_ROOT / 'Drawing_Analysis_By_Type' / '01_Machined_Parts' / '314884_A.png')
SW_JSON_PATH = str(PROJECT_ROOT / 'debug' / 'sw_temp' / '314884.json')
# SW_JSON_PATH = ''  # Leave empty to skip SW comparison

# Assembly context databases
MATING_CONTEXT_PATH = str(PROJECT_ROOT / 'sw_mating_context.json')
MATE_SPECS_PATH = str(PROJECT_ROOT / 'sw_mate_specs.json')
PART_CONTEXT_PATH = str(PROJECT_ROOT / 'vba_extraction_legacy' / 'json_databases' / 'sw_part_context_complete.json')

# Pipeline settings
USE_VLM = False if USE_VISION_PIPELINE else True  # VLM less useful with vision extraction
CONFIDENCE = 0.25      # YOLO detection confidence threshold (YOLO mode only)
OUTPUT_DIR = str(PROJECT_ROOT / 'debug' / ('run_vision' if USE_VISION_PIPELINE else 'run_notebook'))

# OCR quick-win settings (YOLO mode only)
from ai_inspector.config import default_config

if not USE_VISION_PIPELINE:
    default_config.ocr_retry_enabled = True
    default_config.ocr_retry_confidence_threshold = 0.55
    default_config.ocr_retry_max_tokens = 96
    default_config.ocr_retry_max_crop_dimension = 512

# ============================================================
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f'Pipeline mode:   {"VISION (GPT-4o)" if USE_VISION_PIPELINE else "YOLO + OCR"}')
print(f'Image:           {SAMPLE_IMAGE}')
print(f'  exists:        {os.path.exists(SAMPLE_IMAGE)}')
print(f'SW JSON:         {SW_JSON_PATH or "<none>"}')
print(f'  exists:        {os.path.exists(SW_JSON_PATH) if SW_JSON_PATH else "N/A"}')
print(f'Mating context:  {MATING_CONTEXT_PATH or "<none>"}')
print(f'  exists:        {os.path.exists(MATING_CONTEXT_PATH) if MATING_CONTEXT_PATH else "N/A"}')
print(f'Mate specs:      {MATE_SPECS_PATH or "<none>"}')
print(f'  exists:        {os.path.exists(MATE_SPECS_PATH) if MATE_SPECS_PATH else "N/A"}')
print(f'Part context:    {PART_CONTEXT_PATH or "<none>"}')
print(f'  exists:        {os.path.exists(PART_CONTEXT_PATH) if PART_CONTEXT_PATH else "N/A"}')
print(f'VLM:             {"enabled" if USE_VLM else "disabled"}')
print(f'Output dir:      {OUTPUT_DIR}')

In [ ]:
# Cell 3: Import pipeline
from ai_inspector.pipeline.yolo_pipeline import PipelineResult

if USE_VISION_PIPELINE:
    from ai_inspector.pipeline.vision_pipeline import VisionPipeline
    print('VisionPipeline imported successfully (GPT-4o extraction mode).')
else:
    from ai_inspector.pipeline.yolo_pipeline import YOLOPipeline
    print('YOLOPipeline imported successfully (YOLO + OCR mode).')

print(f'PipelineResult fields: {list(PipelineResult.__dataclass_fields__.keys())}')

In [ ]:
# Cell 4: Create pipeline
import torch

def gpu_mem():
    if torch.cuda.is_available():
        a = torch.cuda.memory_allocated() / 1e9
        r = torch.cuda.memory_reserved() / 1e9
        t = torch.cuda.get_device_properties(0).total_memory / 1e9
        return f'{a:.2f} GB allocated / {r:.2f} GB reserved / {t:.1f} GB total'
    return 'No CUDA'

if USE_VISION_PIPELINE:
    # Vision pipeline — no GPU models, just OpenAI API
    pipeline = VisionPipeline(api_key=OPENAI_API_KEY)
    pipeline.load()
    print(f'VisionPipeline ready: {pipeline.is_loaded}')
    print('No GPU models needed — extraction via GPT-4o API')
else:
    # YOLO pipeline — GPU models load/unload sequentially inside run()
    print(f'GPU memory before: {gpu_mem()}')
    pipeline = YOLOPipeline(
        hf_token=HF_TOKEN,
        confidence_threshold=CONFIDENCE,
    )
    pipeline.load()
    print(f'YOLOPipeline ready: {pipeline.is_loaded}')
    print(f'GPU memory after load(): {gpu_mem()}  (no GPU models yet)')

In [ ]:
# Cell 5: Run the full pipeline
import time

mode_str = "VISION (GPT-4o)" if USE_VISION_PIPELINE else "YOLO + OCR"
print(f'Running {mode_str} pipeline on: {os.path.basename(SAMPLE_IMAGE)}')
print(f'VLM: {"ON" if USE_VLM else "OFF"}')
print(f'Mating context: {"ON" if MATING_CONTEXT_PATH else "OFF"}')
print(f'Mate specs: {"ON" if MATE_SPECS_PATH else "OFF"}')
print(f'Part context: {"ON" if PART_CONTEXT_PATH else "OFF"}')
if USE_VISION_PIPELINE:
    print(f'Extraction: GPT-4o vision (single API call)')
else:
    print(f'Sequential loading: YOLO -> unload -> OCR -> unload -> VLM -> unload')
print()

start = time.time()

result = pipeline.run(
    image_path=SAMPLE_IMAGE,
    sw_json_path=SW_JSON_PATH or None,
    page_id='test_page_0',
    output_dir=OUTPUT_DIR,
    save_crops=True,
    use_vlm=USE_VLM,
    mating_context_path=MATING_CONTEXT_PATH or None,
    mate_specs_path=MATE_SPECS_PATH or None,
    part_context_path=PART_CONTEXT_PATH or None,
)

elapsed = time.time() - start

print(f'Pipeline completed in {elapsed:.1f}s')
if not USE_VISION_PIPELINE:
    print(f'GPU memory after run: {gpu_mem()}  (all models unloaded)')
    print(f'Detections: {len(result.packets)}')
else:
    extracted = result.packet_summary.get('extracted_callouts', 0)
    print(f'Extracted callouts: {extracted}  (via GPT-4o vision)')
print(f'Match results: {len(result.match_results)}')
print(f'VLM page understanding: {"yes" if result.page_understanding else "no"}')
print(f'Mating context: {"found" if result.mating_context else "not found"}')
print(f'Mate specs: {"found" if result.mate_specs else "not found"}')

In [ ]:
# Cell 6: VLM Page Understanding results
import json

pu = result.page_understanding
if not pu:
    print('VLM was disabled or returned no results.')
elif pu.get('error'):
    print(f'VLM ERROR: {pu["error"]}')
else:
    print('=== VLM PAGE UNDERSTANDING ===')
    print()

    if pu.get('units'):
        print(f'  Units:         {pu["units"]}')
    if pu.get('drawingType'):
        print(f'  Drawing type:  {pu["drawingType"]}')

    tb = pu.get('titleBlock', {})
    if tb:
        print(f'\n  --- Title Block ---')
        for k, v in tb.items():
            if v:
                print(f'  {k:15s}: {v}')

    tol = pu.get('toleranceBlock', {})
    if tol:
        print(f'\n  --- Default Tolerances ---')
        for k, v in tol.items():
            if v:
                print(f'  {k:15s}: {v}')

    notes = pu.get('generalNotes', [])
    if notes:
        print(f'\n  --- General Notes ({len(notes)}) ---')
        for note in notes:
            print(f'  - {note}')

    sf = pu.get('surfaceFinish', {})
    if sf and sf.get('note'):
        print(f'\n  Surface finish: {sf["note"]}')

    views = pu.get('views', [])
    if views:
        print(f'\n  Views: {", ".join(views)}')

    datums = pu.get('datumReferences', [])
    if datums:
        print(f'  Datums: {", ".join(datums)}')

    print(f'\n  (Full output saved to: {OUTPUT_DIR}/page_understanding.json)')

In [ ]:
# Cell 7: Assembly Mating Context

mc = result.mating_context
if not mc:
    print('No mating context (path not provided or part not found in database).')
else:
    print('=== ASSEMBLY MATING CONTEXT ===')
    print()
    print(f'  Part number:   {mc.get("part_number", "N/A")}')
    print(f'  Description:   {mc.get("description", "N/A")}')
    print(f'  Type:          {mc.get("type", "N/A")}')
    print(f'  Assembly:      {mc.get("assembly", "N/A")}')

    siblings = mc.get('siblings', [])
    if siblings:
        print(f'\n  --- Sibling Components ({len(siblings)}) ---')
        for s in siblings:
            print(f'    {s.get("pn", "?")} — {s.get("desc", "?")} ({s.get("type", "?")})')

    siblings_str = mc.get('siblings_str', '')
    if siblings_str:
        print(f'\n  Siblings summary: {siblings_str}')

    print(f'\n  (Full output saved to: {OUTPUT_DIR}/assembly_context.json)')

In [ ]:
# Cell 8: Mate Specifications (thread/interface constraints)

ms = result.mate_specs
if not ms:
    print('No mate specs (path not provided or part/siblings not found).')
else:
    source = ms.get('source', 'direct')
    print(f'=== MATE SPECIFICATIONS (source: {source}) ===')
    print()

    if source == 'sibling_cross_reference':
        # Specs found via sibling parts
        print(f'  Part {ms.get("part_number")} not directly in mate_specs.')
        print(f'  Showing specs from {len(ms.get("sibling_specs", []))} sibling(s):')
        for spec in ms.get('sibling_specs', []):
            pn = spec.get('part_number', '?')
            desc = spec.get('description', '')
            print(f'\n  --- Sibling: {pn} ({desc}) ---')
            for m in spec.get('mates_with', []):
                line = f'    {m.get("mate_type", "?")} with {m.get("part", "?")}'
                if m.get('description'):
                    line += f' ({m["description"]})'
                if m.get('thread'):
                    line += f'  ** THREAD: {m["thread"]} pitch={m.get("pitch","")} len={m.get("length","")} **'
                print(line)
    else:
        # Direct match
        print(f'  Part: {ms.get("part_number", "?")}')
        print(f'  Description: {ms.get("description", "")}')
        print(f'  Total mates: {ms.get("mate_count", "?")}')
        for m in ms.get('mates_with', []):
            line = f'    {m.get("mate_type", "?")} with {m.get("part", "?")}'
            if m.get('description'):
                line += f' ({m["description"]})'
            if m.get('thread'):
                line += f'  ** THREAD: {m["thread"]} pitch={m.get("pitch","")} len={m.get("length","")} **'
            print(line)

    print(f'\n  (Full output saved to: {OUTPUT_DIR}/mate_specs.json)')

In [ ]:
# Cell 9: Scores, expansion, validation
import json

print('=== SCORES ===')
if result.scores:
    for key, val in result.scores.items():
        print(f'  {key:25s}: {val}')
else:
    print('  (no scores — run without SW data)')

print('\n=== EXPANSION SUMMARY ===')
print(json.dumps(result.expansion_summary, indent=2))

print('\n=== VALIDATION STATS ===')
print(json.dumps(result.validation_stats, indent=2))

print('\n=== PACKET SUMMARY ===')
print(json.dumps(result.packet_summary, indent=2))

In [ ]:
# Cell 10: Match results table
from ai_inspector.comparison.matcher import MatchStatus

if not result.match_results:
    print('No match results (no SW data provided or no features detected).')
else:
    print(f'{"#":>3s} {"Status":15s} {"Type":18s} {"Delta":>10s} {"Notes"}')
    print('=' * 90)

    for i, r in enumerate(result.match_results):
        callout_type = ''
        if r.drawing_callout:
            callout_type = r.drawing_callout.get('calloutType', '')
        elif r.sw_feature:
            callout_type = r.sw_feature.feature_type

        delta_str = f'{r.delta:+.4f}' if r.delta is not None else 'N/A'

        marker = {
            MatchStatus.MATCHED: '[OK]',
            MatchStatus.MISSING: '[MISS]',
            MatchStatus.EXTRA: '[EXTRA]',
            MatchStatus.TOLERANCE_FAIL: '[TOL]',
            MatchStatus.SKIPPED: '[SKIP]',
        }.get(r.status, '[?]')

        print(f'{i:3d} {marker + " " + r.status.value:15s} '
              f'{callout_type:18s} {delta_str:>10s} {r.notes[:50]}')

In [ ]:
# Cell 11: Packet provenance / extracted callouts

if USE_VISION_PIPELINE:
    # Vision pipeline: show extracted callouts from debug file
    import json
    callouts_path = Path(OUTPUT_DIR) / 'callouts_extracted.json'
    if callouts_path.exists():
        with open(callouts_path, 'r', encoding='utf-8') as f:
            callouts = json.load(f)
        print(f'=== GPT-4o Extracted Callouts ({len(callouts)}) ===')
        print()
        for i, c in enumerate(callouts):
            ct = c.get('calloutType', '?')
            raw = c.get('raw', '')[:60]
            qty = c.get('quantity', 1)
            parts = [f'{ct}']
            if c.get('diameter') is not None:
                parts.append(f'dia={c["diameter"]}')
            if c.get('radius') is not None:
                parts.append(f'R={c["radius"]}')
            if c.get('size') is not None:
                parts.append(f'size={c["size"]}')
            if c.get('thread'):
                t = c['thread']
                parts.append(f'thread={t.get("raw", "")}')
            if qty > 1:
                parts.append(f'qty={qty}')
            print(f'  {i+1:2d}. {" | ".join(parts)}')
            print(f'      raw: "{raw}"')
    else:
        print('No callouts_extracted.json found.')
else:
    # YOLO pipeline: show packet provenance
    if not result.packets:
        print('No packets (no detections found in image).')
    else:
        print(f'=== Packet Provenance (first {min(5, len(result.packets))}) ===')
        print()

        for i, pkt in enumerate(result.packets[:5]):
            print(f'--- Packet {i}: {pkt.det_id} ---')

            if pkt.detection:
                print(f'  Detection: class={pkt.detection.class_name}, '
                      f'conf={pkt.detection.confidence:.3f}')

            if pkt.crop:
                meta = pkt.crop.meta or {}
                print(f'  Crop: {meta.get("crop_w", "?")}x{meta.get("crop_h", "?")}px, '
                      f'angle={meta.get("rotation_angle", 0):.1f}deg')

            if pkt.rotation:
                print(f'  Rotation: {pkt.rotation.rotation_used}deg, '
                      f'quality={pkt.rotation.quality_score:.2f}')

            if pkt.reader:
                print(f'  Reader: type={pkt.reader.callout_type}, '
                      f'source={pkt.reader.source}, '
                      f'ocr_conf={pkt.reader.ocr_confidence:.2f}')
                print(f'  Raw OCR: "{pkt.reader.raw[:80]}"')
                if pkt.reader.parsed:
                    parsed_keys = [k for k in pkt.reader.parsed.keys() if not k.startswith('_')]
                    print(f'  Parsed: {dict((k, pkt.reader.parsed[k]) for k in parsed_keys)}')

            if pkt.normalized:
                method = pkt.normalized.get('_normalization_method', '?')
                units = pkt.normalized.get('_detected_units', '?')
                draw_units = pkt.normalized.get('_drawing_units', '?')
                print(f'  Normalization: method={method}, detected={units}, drawing={draw_units}')

            print(f'  Validated: {pkt.validated}'
                  + (f', error="{pkt.validation_error}"' if pkt.validation_error else ''))
            if hasattr(pkt, 'match_status') and pkt.match_status:
                print(f'  Match: {pkt.match_status}')

            print()

In [ ]:
# Cell 12: List saved artifacts
from pathlib import Path

out = Path(OUTPUT_DIR)
print(f'Artifacts saved to: {OUTPUT_DIR}/')
print()

for f in sorted(out.rglob('*')):
    if f.is_file():
        size_kb = f.stat().st_size / 1024
        print(f'  {f.relative_to(out)}  ({size_kb:.1f} KB)')

# Save the full result summary
import json
summary_path = out / 'pipeline_summary.json'
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(result.to_dict(), f, indent=2, ensure_ascii=False)
print(f'\nPipeline summary saved to: {summary_path}')

In [ ]:
# Cell 13: GPT-4o QC Report — full JSON context passed to GPT
import json, os
from pathlib import Path

if not OPENAI_API_KEY:
    print('OPENAI_API_KEY not set — skipping report generation.')
else:
    from ai_inspector.report.qc_report import generate_from_pipeline

    # --- Load extracted callouts from debug output ---
    extracted_callouts = []
    callouts_path = Path(OUTPUT_DIR) / 'callouts_extracted.json'
    if callouts_path.exists():
        with open(callouts_path, 'r', encoding='utf-8') as f:
            extracted_callouts = json.load(f)

    # --- Load validated callouts from debug output ---
    validated_callouts = []
    validated_path = Path(OUTPUT_DIR) / 'validated_callouts.json'
    if validated_path.exists():
        with open(validated_path, 'r', encoding='utf-8') as f:
            validated_callouts = json.load(f)

    # --- Load SW identity (part number, description, material, etc.) ---
    sw_identity = {}
    if SW_JSON_PATH and os.path.exists(SW_JSON_PATH):
        with open(SW_JSON_PATH, 'r', encoding='utf-8-sig') as f:
            sw_raw = json.load(f)
        sw_identity = sw_raw.get('identity', {})

    # --- Generate report with full JSON context ---
    print(f'Part: {sw_identity.get("partNumber", "?")} — {sw_identity.get("description", "?")}')
    print(f'Extracted callouts: {len(extracted_callouts)}')
    print(f'Validated callouts: {len(validated_callouts)}')
    print(f'Match results: {len(result.match_results)}')
    print()
    print('Sending full inspection context as JSON to GPT-4o...')

    report, inspection_context = generate_from_pipeline(
        result=result,
        extracted_callouts=extracted_callouts,
        validated_callouts=validated_callouts,
        sw_identity=sw_identity,
        api_key=OPENAI_API_KEY,
        model='gpt-4o',
    )

    print(f'Report generated — Status: {report.status}')
    print(f'Model: {report.model_used}')
    print()
    print('=' * 70)
    print(report.report_text)
    print('=' * 70)

    # Save report
    report_path = Path(OUTPUT_DIR) / 'qc_report.md'
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(report.report_text)

    # Save full context that was sent to GPT
    context_path = Path(OUTPUT_DIR) / 'inspection_context.json'
    with open(context_path, 'w', encoding='utf-8') as f:
        json.dump(inspection_context, f, indent=2, ensure_ascii=False, default=str)

    # Save structured report data
    report_data_path = Path(OUTPUT_DIR) / 'qc_report.json'
    with open(report_data_path, 'w', encoding='utf-8') as f:
        json.dump(report.to_dict(), f, indent=2, ensure_ascii=False)

    print(f'\nReport saved to: {report_path}')
    print(f'Full context saved to: {context_path}')
    print(f'Report data saved to: {report_data_path}')

In [ ]:
# Cell 14: Cleanup
import gc, torch

pipeline.unload()
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f'GPU memory after cleanup: {gpu_mem()}')
else:
    print('No GPU cleanup needed (vision pipeline is CPU + API only).')
print('Done.')